In [ ]:
from pathlib import Path
import os

In [ ]:
fp = os.path.join(Path().absolute(), "inputs", "input5.txt")

with open(fp, "r") as f:
    data = f.readlines()

In [ ]:
data

# Part 1

In [ ]:
seeds = data[0].strip().split("seeds: ")[1].split()
seeds = [int(x) for x in seeds]

start_indices = {}
end_indices = {}

current = None

maps = ["seed-to-soil", "soil-to-fertilizer", "fertilizer-to-water", "water-to-light", "light-to-temperature", "temperature-to-humidity", "humidity-to-location"]

for i, line in enumerate(data):

    for map in maps:

        if map in line:
            start_indices[map] = i + 1
            current = map
        elif line == "\n":
            end_indices[current] = i - 1

end_indices["humidity-to-location"] = len(data) - 1

In [ ]:
start_indices

In [ ]:
end_indices

In [ ]:
translations = {}
for map in maps:

    translations[map] = []
    for line in data[start_indices[map]:end_indices[map]]:
        dest_range_start, src_range_start, range_len = line.strip().split()
        dest_range_start = int(dest_range_start)
        src_range_start = int(src_range_start)
        range_len = int(range_len)
        src_range_end = src_range_start + range_len - 1
        offset = dest_range_start - src_range_start
        translations[map].append((src_range_start, src_range_end, offset))

In [ ]:
translations

In [ ]:
seeds

In [ ]:
def translate(src_list, map):

    dest_list = [None] * len(src_list)

    for i, src in enumerate(src_list):
        dest = None
        for src_range_start, src_range_end, offset in translations[map]:
            if src_range_start <= src <= src_range_end:
                dest = src + offset
                break

        if dest is None:
            dest = src

        dest_list[i] = dest

    return dest_list

In [ ]:
current_list = seeds
for map in maps:
    print(map)
    current_list = translate(current_list, map)

In [ ]:
current_list

In [ ]:
min(current_list)

# Part 2

In [ ]:
for i in range(int(len(seeds) / 2)):
    start_seed, range_len = seeds[(2 * i) : (2 * i + 2)]
    print(i, range_len / 1e6)

    current_list = range(start_seed, start_seed + 10)
    for map in maps:
        print(map)
        current_list = translate(current_list, map)
        

In [ ]:
sorted_maps = {}
for map in maps:
    print(map)

    sorted_map = sorted(translations[map], key=lambda x: x[0])
    added_brackets = [(-float("inf"), sorted_map[0][0] - 1, 0), (sorted_map[-1][1] + 1, float("inf"), 0)]

    for i in range(len(sorted_map) - 1):
        if sorted_map[i][1] >= sorted_map[i + 1][0]:
            raise ValueError
        elif sorted_map[i + 1][0] - sorted_map[i][1] > 1:
            added_brackets.append((sorted_map[i][1] + 1, sorted_map[i + 1][0] - 1, 0))
    
    sorted_map += added_brackets
    sorted_map = sorted(sorted_map, key=lambda x: x[0])

    sorted_maps[map] = sorted_map

In [ ]:
sorted_maps["seed-to-soil"]

In [ ]:
sorted_maps["soil-to-fertilizer"]

In [ ]:
def combine_maps(sorted_map1, sorted_map2):

    combined_map = []
    for start, end, offset in sorted_map1:
        start_dest = start + offset
        end_dest = end + offset

        # Find which bracket start_dest falls into
        mapped_idx_start = None
        for i, (s, e, o) in enumerate(sorted_map2):
            if s <= start_dest <= e:
                mapped_idx_start = i
                break
        
        # Find which bracket end_dest falls into
        mapped_idx_end = None
        for i, (s, e, o) in enumerate(sorted_map2):
            if s <= end_dest <= e:
                mapped_idx_end = i
                break

        if mapped_idx_start == mapped_idx_end:
            o2 = sorted_map2[mapped_idx_start][2]
            combined_map.append((start, end, offset + o2))
        else:
            for idx in range(mapped_idx_start, mapped_idx_end + 1):
                s2, e2, o2 = sorted_map2[idx]
                if idx == mapped_idx_start:
                    combined_map.append((start, e2 - offset, offset + o2))
                elif idx == mapped_idx_end:
                    combined_map.append((s2 - offset, end, offset + o2))
                else:
                    combined_map.append((s2 - offset, e2 - offset, offset + o2))

    return combined_map

In [ ]:
map1 = sorted_maps["seed-to-soil"]
map2 = sorted_maps["soil-to-fertilizer"]
combined_map = combine_maps(map1, map2)
combined_map

In [ ]:
combined_map = sorted_maps[maps[0]]
for i in range(1, len(maps)):
    map2 = maps[i]
    print(map2)
    sorted_map2 = sorted_maps[map2]
    combined_map = combine_maps(combined_map, sorted_map2)
    print(len(combined_map))

In [ ]:
combined_map

In [ ]:
for i in range(int(len(seeds) / 2)):
    start_seed, range_len = seeds[(2 * i) : (2 * i + 2)]
    print(i, range_len / 1e6)

    current_list = range(start_seed, start_seed + 10)
    for map in maps:
        print(map)
        current_list = translate(current_list, map)
        

In [ ]:
# Build map with zero offset from seeds ranges
seeds_dummy_map = []
for i in range(int(len(seeds) / 2)):
    start_seed, range_len = seeds[(2 * i) : (2 * i + 2)]
    print(i, range_len / 1e6)

    seeds_dummy_map.append((start_seed, start_seed + range_len - 1, 0))

In [ ]:
seeds_dummy_map

In [ ]:
# sorted_map = sorted(seeds_dummy_map, key=lambda x: x[0])
# added_brackets = [(-float("inf"), sorted_map[0][0] - 1, 0), (sorted_map[-1][1] + 1, float("inf"), 0)]

# for i in range(len(sorted_map) - 1):
#     if sorted_map[i][1] >= sorted_map[i + 1][0]:
#         raise ValueError
#     elif sorted_map[i + 1][0] - sorted_map[i][1] > 1:
#         added_brackets.append((sorted_map[i][1] + 1, sorted_map[i + 1][0] - 1, 0))

# sorted_map += added_brackets
# sorted_map = sorted(sorted_map, key=lambda x: x[0])
# seeds_dummy_map = sorted_map
# seeds_dummy_map

In [ ]:
# Then combine w combined map
final_map = combine_maps(seeds_dummy_map, combined_map)
final_map

In [ ]:
# Then compute start + offset of each range
candidates = [s + o for s, e, o in final_map]

# pick start giving min

In [ ]:
min_elt = min(final_map, key=lambda x: x[0] + x[2])
min_elt

In [ ]:
min_elt[0] + min_elt[2]